# Particle tagger study at DarkQuest
**Author:** Dowling Wong

This is the demo on how to use the DNN based multi-class tagger for DarkQuest. In this demo, we are going to stick with dwong.py, the integrated python module for data process and tagging. Please refer to [dwong.py](dwong.py) for detailed implementation of code.
## Clustering
After reconstruct hits coordinates on EMCAl, we have performed seed-searching algorithm and K-Means clustering. Since Kmeans clustering doesnot counted energy for each hits, centroid may change exceed tolerance during iterations. Return coords, labels, energy, seeds(centroids), in energy decrease order.  
## Link algorithm
Using tracklet information of station2&3, we perform linear extrapolation to EMCal and station4 hodoscope. By tolerace distance 8cm(for most particles, this value should under 5cm), and assign closest unique track to every h4 hit. 

## DNN taggers
Our DNN Multiclass tagger has 4 binary classifiers:  
-[Electron/Positron](NNs/electronID_w_track_95), with AUC 0.95   
-[Muon](NNs/muonID_w_track_99), with AUC 0.99  
-[Muon](NNs/photonID_w_track_89), with AUC 0.89  
-[Pi+/Pi-](NNs/pi+-_ID_w_track_86), with AUC 0.86  
AUC gained with equal weight mixture of electron, positron, photon/Pi0, Pi+, Pi-, Klong. Assign particle tag by inspecting array of outputs by NNs.


## Py-ROOT binding


In [2]:
import dwong

In [2]:
from keras.models import load_model
import numpy as np

NN_dir="NNs/"

Electron_ID = load_model(NN_dir+"electronID_w_track_95")
Muon_ID = load_model(NN_dir+"muonID_w_track_99")
Pion_ID = load_model(NN_dir+"pi+-_ID_w_track_86")
Photon_ID = load_model(NN_dir+"photon_ID_w_track_89")


def predict_classes(samples):
    probs_e = Electron_ID.predict(samples)
    probs_m = Muon_ID.predict(samples)
    probs_pi = Pion_ID.predict(samples)
    probs_ph = Photon_ID.predict(samples)

    # Combine predictions for each sample
    combined_probs = np.stack([probs_e, probs_m, probs_pi, probs_ph], axis=-1)
    class_indices = np.argmax(combined_probs, axis=-1)

    # Map indices to class names
    classes = ['electron', 'muon', 'pion', 'photon']
    predicted_classes = [classes[idx] for idx in class_indices]

    return predicted_classes


In [ ]:
num_samples = 100000  # Number of particles
input_shape = (shape_of_your_input,)  # Replace with the shape of your model input
samples = np.random.rand(num_samples, *input_shape)  # Generate or load your samples
predicted_classes = predict_classes(samples)

# Do something with predicted_classes, like analyzing or saving them
